# Model Evaluation

## Four-fold Cross Evaluation

```
Fold 1: trained on FRH01, FRH02, FRH03, tested on FRH04
Fold 2: trained on FRH01, FRH02, FRH04, tested on FRH03
Fold 3: trained on FRH01, FRH03, FRH04, tested on FRH02
Fold 4: trained on FRH02, FRH03, FRH04, tested on FRH01
```

## Download and unzip evaluation runs
to `/tmp`


Folder structure
```
<level>/
    <fold>/
        <model>/
            y_pred.npy
            y_true.npy
            classification_report.txt
            model.pth
            ...
```

In [2]:
!wget -N -P /tmp https://syncandshare.lrz.de/dl/fi6zpj2xQ24mM3VxLewj87eQ/L1C.zip
!unzip -o /tmp/L1C.zip -d /tmp

!wget -N -P /tmp https://syncandshare.lrz.de/dl/fi5D238TjzBYFr78ERmeUbCd/L2A.zip    
!unzip -o /tmp/L2A.zip -d /tmp

--2022-12-21 13:17:21--  https://syncandshare.lrz.de/dl/fi6zpj2xQ24mM3VxLewj87eQ/L1C.zip
Resolving syncandshare.lrz.de (syncandshare.lrz.de)... ::ffff:129.187.255.213, 129.187.255.213
Connecting to syncandshare.lrz.de (syncandshare.lrz.de)|::ffff:129.187.255.213|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-12-21 13:17:22 ERROR 404: Not Found.

/bin/bash: unzip: command not found
--2022-12-21 13:17:22--  https://syncandshare.lrz.de/dl/fi5D238TjzBYFr78ERmeUbCd/L2A.zip
Resolving syncandshare.lrz.de (syncandshare.lrz.de)... ::ffff:129.187.255.213, 129.187.255.213
Connecting to syncandshare.lrz.de (syncandshare.lrz.de)|::ffff:129.187.255.213|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-12-21 13:17:22 ERROR 404: Not Found.

/bin/bash: unzip: command not found


In [5]:
from breizhcrops.models.pretrained import pretrained

In [6]:
transformer = pretrained(model="transformer")

TransformerEncoder_input-dim%3D13_num-classes%3D9_d-model%3D64_d-inner%3D128_n-layers%3D5_n-head%3D2_dropout%3D0.017998950510888446_learning-rate%3D0.00017369201853408445_weight-decay%3D3.5156458637523697e-06.pth: 705kB [00:00, 1.32MB/s]                            


In [13]:
transformer.state_dict()


OrderedDict([('inlinear.weight',
              tensor([[ 1.0950e-39, -1.5410e-22, -5.5890e-20, -1.3366e-39,  5.6565e-40,
                       -3.8686e-37, -1.9053e-39, -9.8917e-40, -2.7324e-41, -1.9134e-39,
                       -7.8348e-40, -8.3649e-27, -9.9177e-22],
                      [-2.6271e-40,  5.3684e-40, -1.1668e-39,  1.9357e-39,  2.7136e-40,
                       -9.6071e-40,  2.1720e-43,  1.9151e-39,  1.1955e-39, -1.2330e-39,
                       -1.3816e-39, -1.1987e-39, -1.8853e-39],
                      [-2.9285e-01, -4.5158e-02,  1.8766e-01,  2.7250e-01,  2.7254e-01,
                       -1.5665e-01,  2.1866e-01, -2.4550e-01,  1.5848e-01, -4.9148e-02,
                       -1.5939e-01, -1.8607e-01, -6.7024e-02],
                      [ 6.7172e-02,  1.7126e-01,  1.5430e-01, -4.0058e-01, -3.1641e-01,
                       -2.2003e-02, -1.6171e-01, -3.2863e-01, -1.0586e-01,  2.3327e-01,
                        7.8138e-02,  1.4692e-01,  6.6440e-02],
           

In [17]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import sys
sys.path.append("..")
from examples.train import metrics as calc_metrics

def load(run):
    y_pred = np.load(os.path.join(run,"y_pred.npy"))
    y_true = np.load(os.path.join(run,"y_true.npy"))
    y_score = np.load(os.path.join(run,"y_score.npy"))
    field_ids = np.load(os.path.join(run,"field_ids.npy"))
    rs = pd.DataFrame([y_pred,y_true,field_ids],index=["y_pred","y_true","field_ids"]).T.set_index("field_ids")
    return rs, y_score

def load_table(logdir):
    runs = os.listdir(logdir)
    #runs = ["LSTM","OmniScaleCNN","MSResNet","StarRNN","TempCNN","TransformerEncoder", "InceptionTime"]

    stats = list()
    for run in runs:
        rs, _ = load(os.path.join(logdir,run))
        stat = calc_metrics(rs.y_true,rs.y_pred)
        stat["model"] = run
        stats.append(stat)
    stats = pd.DataFrame(stats).set_index("model")

    df = stats.T
    #df["RF"] = ""

    #models = ["RF","OmniScaleCNN", "TempCNN","MSResNet", "InceptionTime", "LSTM","StarRNN","TransformerEncoder"]
    table = df.loc[["accuracy","recall_macro","f1_macro","f1_weighted","kappa"]]
    table.index = ["overall accuracy","class-mean accuracy","class-mean f-score","weighted f-score","kappa-metric"]
    return table

### Compile L1C Results

In [43]:
logdir = "/workspaces/BreizhCrops/results/serbia"

l1tables = []
for fold in [1,2,3,4]:
    table = load_table(f"{logdir}/{fold}").T
    table["fold"] = fold
    l1tables.append(table)


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

In [44]:
meantable = pd.concat(l1tables).groupby(["model"]).mean()
stdtable = pd.concat(l1tables).groupby(["model"]).std()

metrics = ["overall accuracy","class-mean accuracy","class-mean f-score","weighted f-score","kappa-metric"]
models = ["TransformerEncoder", "TransformerEncoderRF", "TransformerPretrainedFE", "TransformerPretrainedFT", "TransformerPretrainedIN"]
for metric in metrics:
    entries = list()
        
    for model in models:
        entries.append("$" + f"{meantable.loc[model,metric]:.2f}"+"^{\\pm "+ f"{stdtable.loc[model,metric]:.2f}" + "}$")
        row = f"{metric} & & " + " & ".join(entries)
        row += " \\\ "
        print(row)

overall accuracy & & $0.92^{\pm 0.01}$ \\ 
overall accuracy & & $0.92^{\pm 0.01}$ & $0.82^{\pm 0.01}$ \\ 
overall accuracy & & $0.92^{\pm 0.01}$ & $0.82^{\pm 0.01}$ & $0.94^{\pm 0.01}$ \\ 
overall accuracy & & $0.92^{\pm 0.01}$ & $0.82^{\pm 0.01}$ & $0.94^{\pm 0.01}$ & $0.94^{\pm 0.01}$ \\ 
overall accuracy & & $0.92^{\pm 0.01}$ & $0.82^{\pm 0.01}$ & $0.94^{\pm 0.01}$ & $0.94^{\pm 0.01}$ & $0.94^{\pm 0.01}$ \\ 
class-mean accuracy & & $0.77^{\pm 0.03}$ \\ 
class-mean accuracy & & $0.77^{\pm 0.03}$ & $0.59^{\pm 0.02}$ \\ 
class-mean accuracy & & $0.77^{\pm 0.03}$ & $0.59^{\pm 0.02}$ & $0.89^{\pm 0.02}$ \\ 
class-mean accuracy & & $0.77^{\pm 0.03}$ & $0.59^{\pm 0.02}$ & $0.89^{\pm 0.02}$ & $0.91^{\pm 0.02}$ \\ 
class-mean accuracy & & $0.77^{\pm 0.03}$ & $0.59^{\pm 0.02}$ & $0.89^{\pm 0.02}$ & $0.91^{\pm 0.02}$ & $0.90^{\pm 0.03}$ \\ 
class-mean f-score & & $0.76^{\pm 0.02}$ \\ 
class-mean f-score & & $0.76^{\pm 0.02}$ & $0.64^{\pm 0.03}$ \\ 
class-mean f-score & & $0.76^{\pm 0.02}$ & $0

In [45]:
pd.concat(l1tables).groupby(["model","fold"]).first()

overall accuracy  class-mean accuracy  \
model                   fold                                          
TransformerEncoder      1             0.907357             0.722419   
                        2             0.940000             0.800572   
                        3             0.923706             0.774803   
                        4             0.926364             0.771472   
TransformerEncoderRF    1             0.811081             0.601439   
                        2             0.827273             0.610184   
                        3             0.826521             0.578473   
                        4             0.829091             0.556375   
TransformerPretrainedFE 1             0.925522             0.876171   
                        2             0.942727             0.904892   
                        3             0.938238             0.876753   
                        4             0.937273             0.918559   
TransformerPretrainedFT 1             0.927339             0.878851   
                        2             0.940909             0.930881   
                        3             0.939146             0.913398   
                        4             0.950909             0.902971   
TransformerPretrainedIN 1             0.928247             0.875042   
                        2             0.944545             0.934858   
                        3             0.942779             0.913224   
                        4             0.941818             0.864954   

                              class-mean f-score  weighted f-score  \
model                   fold                                         
TransformerEncoder      1               0.736774          0.904964   
                        2               0.792621          0.938577   
                        3               0.767335          0.922537   
                        4               0.760506          0.925082   
TransformerEncoderRF    1               0.674465          0.799234   
                        2               0.650107          0.815562   
                        3               0.634881          0.816632   
                        4               0.602426          0.816843   
TransformerPretrainedFE 1               0.888639          0.925426   
                        2               0.918502          0.942554   
                        3               0.862830          0.938303   
                        4               0.909204          0.937632   
TransformerPretrainedFT 1               0.895594          0.927180   
                        2               0.930882          0.940727   
                        3               0.886091          0.938690   
                        4               0.907149          0.951034   
TransformerPretrainedIN 1               0.894015          0.927989   
                        2               0.934559          0.944353   
                        3               0.905160          0.942449   
                        4               0.878737          0.941822   

                              kappa-metric  
model                   fold                
TransformerEncoder      1         0.862451  
                        2         0.912025  
                        3         0.887861  
                        4         0.891508  
TransformerEncoderRF    1         0.708566  
                        2         0.736926  
                        3         0.732941  
                        4         0.736986  
TransformerPretrainedFE 1         0.890677  
                        2         0.915669  
                        3         0.909329  
                        4         0.908655  
TransformerPretrainedFT 1         0.893384  
                        2         0.913171  
                        3         0.910062  
                        4         0.927963  
TransformerPretrainedIN 1         0.893969  
                        2         0.918304  
                        3 

- TransformerEncoderFull - Train on the vojvodina dataset (without transfer)
- TransformerEncoderRF - FE + Random Forest
- TransformerPretrainedFT - Fin Tuning (transformer layer + linear output layer tuned)
- TransformerPretrainedFE - Feature Extraction (only output linear layer is tuned)


In [7]:
l2tables = []
for fold in [1,2,3,4]:
    table = load_table(f"{logdir}/L2A/{fold}").T
    table["fold"] = fold
    l2tables.append(table)

FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/BreizhCrops/results/france/L2A/1'

In [11]:

meantable = pd.concat(l2tables).groupby(["model"]).mean()
stdtable = pd.concat(l2tables).groupby(["model"]).std()

metrics = ["overall accuracy","average accuracy","class-mean f-score","weighted f-score","kappa-metric"]
models = ["OmniScaleCNN","TempCNN","MSResNet", "InceptionTime", "LSTM", "StarRNN", "TransformerEncoder"]
for metric in metrics:
    entries = list()
        
    for model in models:
        entries.append("$" + f"{meantable.loc[model,metric]:.2f}"+"^{\\pm "+ f"{stdtable.loc[model,metric]:.2f}" + "}$")
    row = f"{metric} & & " + " & ".join(entries)
    row += " \\\ "
    print(row)

overall accuracy & & $0.75^{\pm 0.06}$ & $0.79^{\pm 0.03}$ & $0.76^{\pm 0.02}$ & $0.73^{\pm 0.03}$ & $0.78^{\pm 0.05}$ & $0.78^{\pm 0.04}$ & $0.79^{\pm 0.03}$ \\ 
average accuracy & & $0.54^{\pm 0.04}$ & $0.55^{\pm 0.04}$ & $0.55^{\pm 0.04}$ & $0.51^{\pm 0.02}$ & $0.56^{\pm 0.03}$ & $0.55^{\pm 0.02}$ & $0.58^{\pm 0.03}$ \\ 
class-mean f-score & & $0.54^{\pm 0.04}$ & $0.55^{\pm 0.03}$ & $0.57^{\pm 0.04}$ & $0.49^{\pm 0.03}$ & $0.56^{\pm 0.03}$ & $0.56^{\pm 0.02}$ & $0.58^{\pm 0.02}$ \\ 
weighted f-score & & $0.73^{\pm 0.06}$ & $0.78^{\pm 0.04}$ & $0.76^{\pm 0.02}$ & $0.70^{\pm 0.06}$ & $0.78^{\pm 0.05}$ & $0.78^{\pm 0.04}$ & $0.79^{\pm 0.04}$ \\ 
kappa-metric & & $0.68^{\pm 0.08}$ & $0.72^{\pm 0.05}$ & $0.70^{\pm 0.03}$ & $0.65^{\pm 0.04}$ & $0.72^{\pm 0.06}$ & $0.72^{\pm 0.05}$ & $0.73^{\pm 0.05}$ \\ 


In [13]:
pd.concat(l2tables).groupby(["model","fold"]).first()

overall accuracy  average accuracy  \
model              fold                                       
InceptionTime      1             0.771050          0.533800   
                   2             0.730403          0.476804   
                   3             0.693775          0.523857   
                   4             0.734605          0.507277   
LSTM               1             0.800983          0.566342   
                   2             0.722138          0.517356   
                   3             0.781208          0.562395   
                   4             0.829336          0.580922   
MSResNet           1             0.766837          0.536133   
                   2             0.742245          0.508646   
                   3             0.786983          0.572476   
                   4             0.750512          0.598012   
OmniScaleCNN       1             0.734671          0.519379   
                   2             0.681896          0.500232   
                   3             0.760573          0.556235   
                   4             0.825972          0.583364   
StarRNN            1             0.789313          0.555245   
                   2             0.734545          0.515473   
                   3             0.797581          0.563910   
                   4             0.817591          0.560113   
TempCNN            1             0.794830          0.554077   
                   2             0.734118          0.497686   
                   3             0.803199          0.571277   
                   4             0.809609          0.575472   
TransformerEncoder 1             0.804552          0.584775   
                   2             0.743519          0.541460   
                   3             0.802638          0.582379   
                   4             0.823957          0.606289   

                         class-mean f-score  weighted f-score  kappa-metric  
model              fold                                                      
InceptionTime      1               0.536306          0.766904      0.703828  
                   2               0.502645          0.717395      0.643124  
                   3               0.462870          0.614834      0.603513  
                   4               0.471220          0.701496      0.659076  
LSTM               1               0.576579          0.799199      0.739637  
                   2               0.527994          0.720954      0.639460  
                   3               0.560205          0.764094      0.714349  
                   4               0.587238          0.827328      0.782179  
MSResNet           1               0.549065          0.769380      0.697171  
                   2               0.531518          0.737054      0.660065  
                   3               0.570622          0.769226      0.722196  
                   4               0.621109          0.746279      0.704581  
OmniScaleCNN       1               0.513839          0.716275      0.649812  
                   2               0.497293          0.670303      0.592605  
                   3               0.550792          0.726619      0.687063  
                   4               0.593317          0.819853      0.776827  
StarRNN            1               0.561890          0.789067      0.727189  
                   2               0.521541          0.732364      0.649706  
                   3               0.568356          0.792801      0.736649  
                   4               0.573021          0.815582      0.766583  
TempCNN            1               0.561041          0.787383      0.730600  
                   2               0.507293          0.728791      0.650568  
                   3               0.573574          0.799311      0.743531  
                   4               0.573071          0.810186      0.759289  
TransformerEncoder 1               0.590879          0.803417      0.745349  
                   2        